In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error

In [2]:
train_file = pd.read_csv('../input/train.csv')
y_true = train_file.deal_probability.values
baseline = np.zeros(len(y_true))

In [4]:
# First oof then test
to_blend = [('../blender/train_nima_preds.csv', '../blender/test_nima_preds.csv', 'nima_preds'),
            ('../blender/lgbm_cv_5folds final.csv', '../blender/submissionLGBCV5foldsfinal.csv', 'deal_probability'),
            ('../blender/oof_train.csv', '../blender/oof_test.csv', 'fm_2204'),
            ('../blender/oof_train.csv', '../blender/oof_test.csv', 'lgbm_nonima_2195'),
            ('../blender/oof_train.csv', '../blender/oof_test.csv', 'rnn_2206'),
            ('../blender/oof_train.csv', '../blender/oof_test.csv', 'lgbm_tam_2210'),
            ('../blender/oof_train.csv', '../blender/oof_test.csv', 'lgbm_nima_noridge_2203'),
            ('../blender/oof_fmftrl_5seed.csv', '../blender/fmftrl_5seed.csv', 'deal_probability'),
            ('../blender/lgbm_cv_avg10folds.csv', '../blender/submissionLGBCV10folds.csv', 'deal_probability')
           ]

In [8]:
subs = dict()
for train, test, target_name in to_blend:
    train_file = pd.read_csv(train)
    test_file  = pd.read_csv(test)
    label = train.split('/')[-1].replace(' ','_') + '_' + target_name
    subs[label] = dict()
    subs[label]['train'] = train_file[target_name].values
    subs[label]['test'] = test_file[target_name].values

In [5]:
def evaluate_blend(baseline, weight):
    scores = list()
    for submission in subs:
        blend = (1 - weight) * baseline + weight * subs[submission]['train']
        scores.append([np.sqrt(mean_squared_error(y_true, blend)), submission])
    score, label = sorted(scores)[0]
    return score, label

In [7]:
max_count = 1000
count = 0
step = 0.01

model_weights = {s: 0.0 for s in subs}

print('first step')
score, label = evaluate_blend(baseline, weight = 1.0)
best_score = score
baseline = subs[label]['train']
submission = subs[label]['test']
print('starting from model %s with rmse %0.6f' % (label, score))
model_weights[label] = 1.0

while count < max_count:
    score, label = evaluate_blend(baseline, weight = step)
    print ('At step %i the best score is %0.6f adding model %s'% (count+1, score, label))
    if score >= best_score:
        print('ATTENTION: %0.6f is not an improvement of %0.6f --> STOPPING' % (score, best_score))
        break
    else:
        baseline   = (1 - step) * baseline   + step * subs[label]['train']
        submission = (1 - step) * submission + step * subs[label]['test']
        best_score = score
        model_weights = {w:model_weights[w] * (1 - step) for w in model_weights}
        model_weights[label] += step
    count += 1

print ('\nFinal model score: %0.6f' % best_score)
print ('Blend composition:')
for w in model_weights:
    if model_weights[w] > 0:
        print('%s : %0.3f' % (w, model_weights[w]))

first step
starting from model oof_train.csv_lgbm_nima_noridge_2203 with rmse 0.214875
At step 1 the best score is 0.214799 adding model oof_train.csv_fm_2204
At step 2 the best score is 0.214725 adding model oof_train.csv_fm_2204
At step 3 the best score is 0.214654 adding model oof_train.csv_fm_2204
At step 4 the best score is 0.214585 adding model oof_train.csv_fm_2204
At step 5 the best score is 0.214518 adding model oof_train.csv_fm_2204
At step 6 the best score is 0.214454 adding model oof_train.csv_fm_2204
At step 7 the best score is 0.214392 adding model oof_train.csv_fm_2204
At step 8 the best score is 0.214332 adding model oof_train.csv_fm_2204
At step 9 the best score is 0.214274 adding model oof_train.csv_fm_2204
At step 10 the best score is 0.214218 adding model oof_train.csv_fm_2204
At step 11 the best score is 0.214164 adding model oof_train.csv_fm_2204
At step 12 the best score is 0.214112 adding model oof_train.csv_fm_2204
At step 13 the best score is 0.214062 adding m

At step 107 the best score is 0.212665 adding model oof_train.csv_fm_2204
At step 108 the best score is 0.212662 adding model lgbm_cv_5folds_final.csv_deal_probability
At step 109 the best score is 0.212660 adding model oof_train.csv_lgbm_tam_2210
At step 110 the best score is 0.212658 adding model oof_train.csv_fm_2204
At step 111 the best score is 0.212656 adding model lgbm_cv_5folds_final.csv_deal_probability
At step 112 the best score is 0.212655 adding model lgbm_cv_5folds_final.csv_deal_probability
At step 113 the best score is 0.212653 adding model oof_train.csv_fm_2204
At step 114 the best score is 0.212651 adding model oof_train.csv_lgbm_tam_2210
At step 115 the best score is 0.212649 adding model oof_train.csv_rnn_2206
At step 116 the best score is 0.212648 adding model lgbm_cv_5folds_final.csv_deal_probability
At step 117 the best score is 0.212646 adding model oof_train.csv_fm_2204
At step 118 the best score is 0.212645 adding model lgbm_cv_5folds_final.csv_deal_probability